# [Advent of Code 2019](https://adventofcode.com/2019/) - Day 3

## Setup

In [ ]:
def read_input(filename):
    with open(filename) as file:
        input = file.readlines()
    return input

### Part 1

In [ ]:
class Coord:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
        
    def distance(self):
        return abs(self.x) + abs(self.y)

    
class Segment:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def crosses(self, a_segment):
        if self.parallel_to(a_segment):
            return False
        
        if self.is_horizontal():
            h_segment = self
            v_segment = a_segment
        else:
            h_segment = a_segment
            v_segment = self
        
        return (
            (
                (v_segment.minimum().y <= h_segment.start.y) and 
                (h_segment.start.y <= v_segment.maximum().y) 
            ) and ( 
                (h_segment.minimum().x <= v_segment.start.x) and 
                (v_segment.start.x <= h_segment.maximum().x)
            )
        )

    def parallel_to(self, a_segment):
        return (
            (self.is_horizontal() and a_segment.is_horizontal()) or
            (self.is_vertical() and a_segment.is_vertical())
        )

    def is_horizontal(self):
        return self.start.y == self.end.y
 
    def is_vertical(self):
        return self.start.x == self.end.x
    
    def minimum(self):
        return Coord(min(self.start.x, self.end.x), min(self.start.y, self.end.y))
    
    def maximum(self):
        return Coord(max(self.start.x, self.end.x), max(self.start.y, self.end.y))
    
    def crossing_coord(self, a_segment):
        if self.is_horizontal():
            h_segment = self
            v_segment = a_segment
        else:
            h_segment = a_segment
            v_segment = self
            
        return Coord(v_segment.end.x, h_segment.end.y)
        

class Wire:
    def __init__(self, definition):
        self.definition = definition
        self.segments = self._segments()
    
    def _segments(self):
        segments = []
        
        for path in self.definition.split(","):
            if len(segments) == 0:
                last_coord = Coord(0, 0)
            else:
                last_coord = segments[-1].end
    
            next_coord = self._next_coord(last_coord, path)
            segments.append(Segment(last_coord, next_coord))

        return segments
    
    def _next_coord(self, last_coord, path):
        direction = path[0]
        distance = int(path[1:])
            
        if (direction == "L"):
            next_coord = Coord(last_coord.x - distance, last_coord.y) 
        elif (direction == "R"):
            next_coord = Coord(last_coord.x + distance, last_coord.y) 
        elif (direction == "U"):
            next_coord = Coord(last_coord.x, last_coord.y + distance) 
        elif (direction == "D"):
            next_coord = Coord(last_coord.x, last_coord.y - distance) 
        else:
            print(f"Error: Unknown direction encountered, '{direction}'")
            next_coord = None
        
        return next_coord
        
    def intersections(self, wire):
        intersection_coords = []
        
        for segment in self.segments:
            for a_segment in wire.segments[1:]:
                if segment.crosses(a_segment):
                    coord = segment.crossing_coord(a_segment)
                    intersection_coords.append(coord)
        
        return intersection_coords
    
    def steps_to_coord(self, coord):
        index = segment_containing_coord(coord)
        

In [ ]:
def minimum_intersection_distance(wire_1, wire_2):
    minimum_distance = float("inf")
    intersection_coords = wire_1.intersections(wire_2)
    distances = [coord.distance() for coord in intersection_coords]
    minimum_distance = min(minimum_distance, min(distances))

    return minimum_distance    

#### Test

In [ ]:
assert minimum_intersection_distance(
    Wire("R8,U5,L5,D3"),
    Wire("U7,R6,D4,L4"),
) == 6
assert minimum_intersection_distance(
    Wire("R75,D30,R83,U83,L12,D49,R71,U7,L72"),
    Wire("U62,R66,U55,R34,D71,R55,D58,R83"),
) == 159
assert minimum_intersection_distance(
    Wire("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51"),
    Wire("U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"),
) == 135

#### Input

In [ ]:
input_wires = read_input("../../data/2019/03.txt")
wire_1 = Wire(input_wires[0])
wire_2 = Wire(input_wires[1])

print(f"Minimum intersection distance = {minimum_intersection_distance(wire_1, wire_2)}")